In [ ]:
import time

import dask.dataframe as dd
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from ares import ALL_STRUCTURES
from sc2.dicts.unit_tech_alias import UNIT_TECH_ALIAS
from sc2.dicts.unit_trained_from import UNIT_TRAINED_FROM
from sc2.dicts.unit_unit_alias import UNIT_UNIT_ALIAS
from sc2.ids.unit_typeid import UnitTypeId
from torch import optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm.notebook import tqdm

In [25]:
unit_types = ALL_STRUCTURES | UNIT_TRAINED_FROM.keys() | UNIT_TECH_ALIAS.keys() | UNIT_UNIT_ALIAS.keys()
unit_type_values = list(set(u.value for u in unit_types))

In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [27]:
def get_alias(unit_type: UnitTypeId) -> UnitTypeId | None:
    if unit_type == UnitTypeId.HIVE:
        return UnitTypeId.LAIR
    elif aliases := UNIT_TECH_ALIAS.get(unit_type):
        if len(aliases) == 1:
            return next(iter(aliases))
        raise NotImplementedError()
    elif alias := UNIT_UNIT_ALIAS.get(unit_type):
        return alias
    return None


aliases = list((k, v) for k in reversed(UnitTypeId) if (v := get_alias(k)) is not None)

In [28]:
def process(df: pd.DataFrame) -> dd.DataFrame:
    return (
        df.pivot_table(index=["replay_name", "game_loop", "player"], columns=["is_own", "unit_type"], values="count")
        .fillna(0)
        .astype(int)
    )

In [29]:
df_train = pd.read_parquet("../resources/datasets/scout/train.parquet")
df_test = pd.read_parquet("../resources/datasets/scout/test.parquet")

In [30]:
def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    df = df.assign(is_own=df["owner"] == df["player"])
    df = df.pivot_table(
        index=["replay_name", "game_loop", "player", "is_own"], columns=["unit_type"], values="count", fill_value=0
    )
    for alias_from, alias_to in aliases:
        if alias_from.value in df.columns:
            df = df.add(df[alias_from.value].to_frame(name=alias_to.value), fill_value=0)
    df = df.reindex(columns=unit_type_values, fill_value=0)
    return df


def to_numpy(df: pd.DataFrame) -> tuple[np.ndarray, np.ndarray]:
    x1 = df.loc[:, :, 1, :].to_numpy()
    x2 = df.loc[:, :, 2, :].to_numpy()
    x = np.concatenate([x1, x2])
    y = np.concatenate([x2, x1])
    return x, y

In [31]:
df_train

,game_loop,unit_type,player,owner,count,replay_name
0,0,86,1,1,1,3901860_Sharkling_PhantomBot_EphemeronAIE.SC2R...
1,0,86,2,2,1,3901860_Sharkling_PhantomBot_EphemeronAIE.SC2R...
2,0,104,1,1,12,3901860_Sharkling_PhantomBot_EphemeronAIE.SC2R...
3,0,104,2,2,12,3901860_Sharkling_PhantomBot_EphemeronAIE.SC2R...
4,0,106,1,1,1,3901860_Sharkling_PhantomBot_EphemeronAIE.SC2R...
...,...,...,...,...,...,...
25797105,9153,106,1,1,11,3905917_Eris_SharpenedEdge_InterloperAIE.SC2Re...
25797106,9153,126,1,1,8,3905917_Eris_SharpenedEdge_InterloperAIE.SC2Re...
25797107,9153,137,1,1,38,3905917_Eris_SharpenedEdge_InterloperAIE.SC2Re...
25797108,9153,138,1,1,1,3905917_Eris_SharpenedEdge_InterloperAIE.SC2Re...


In [32]:
x_train, y_train = to_numpy(preprocess(df_train))
x_test, y_test = to_numpy(preprocess(df_test))

In [33]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super().__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Linear(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

In [34]:
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super().__init__()
        self.embedding = nn.Linear(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        output, hidden = self.gru(encoder_outputs, encoder_hidden)
        return self.out(output), hidden, None

    # def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
    #     batch_size = encoder_outputs.size(0)
    #
    #     decoder_input = torch.zeros(batch_size, 1, 273, device=device)
    #     decoder_hidden = encoder_hidden
    #     decoder_outputs = []
    #
    #     for i in range(encoder_outputs.size(1)):
    #
    #         decoder_output, decoder_hidden  = self.forward_step(decoder_input, decoder_hidden)
    #         decoder_outputs.append(decoder_output)
    #
    #         if target_tensor is not None:
    #             # Teacher forcing: Feed the target as the next input
    #             decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
    #         else:
    #             # Without teacher forcing: use its own predictions as the next input
    #             _, topi = decoder_output.topk(1)
    #             decoder_input = topi.squeeze(-1).detach()  # detach from history as input
    #
    #     decoder_outputs = torch.cat(decoder_outputs, dim=1)
    #     # decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
    #     return decoder_outputs, decoder_hidden, None # We return `None` for consistency in the training loop
    #
    # def forward_step(self, input, hidden):
    #     output = self.embedding(input)
    #     output = F.relu(output)
    #     output, hidden = self.gru(output, hidden)
    #     output = self.out(output)
    #     return output, hidden

In [35]:
def train_epoch(dataloader, model, optimizer, criterion):
    total_loss = 0
    for data in dataloader:
        input_tensor, target_tensor = data

        optimizer.zero_grad()

        prediction = model(input_tensor)
        loss = criterion(
            prediction,
            target_tensor,
            # decoder_outputs.view(-1, decoder_outputs.size(-1)),
            # target_tensor.view(-1)
        )
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(dataloader)

In [36]:
def test_epoch(dataloader, model, criterion):
    total_loss = 0
    for data in tqdm(dataloader):
        input_tensor, target_tensor = data

        prediction = model(input_tensor)

        loss = criterion(
            prediction,
            target_tensor,
            # decoder_outputs.view(-1, decoder_outputs.size(-1)),
            # target_tensor.view(-1)
        )
        loss.backward()

        total_loss += loss.item()

    return total_loss / len(dataloader)

In [37]:
dataset_train = TensorDataset(torch.Tensor(x_train), torch.Tensor(y_train))
dataset_test = TensorDataset(torch.Tensor(x_test), torch.Tensor(y_test))

In [38]:
dataset_train[-1]

(tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.]),
 tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  

In [39]:
dataloader_train = DataLoader(dataset_train, batch_size=64, shuffle=True)
dataloader_test = DataLoader(dataset_test, batch_size=64, shuffle=True)

In [40]:
def train(train_dataloader, model, n_epochs, learning_rate=0.001):
    time.time()

    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()

    for _epoch in range(1, n_epochs + 1):
        loss = train_epoch(train_dataloader, model, optimizer, criterion)
        print(loss)

In [41]:
input_size = x_train.shape[1]
output_size = y_train.shape[1]
batch_size = 32

model = nn.Linear(input_size, output_size)

train(dataloader_train, model, 10)

25.76875088008031
23.10143742271563
22.639423123649337
22.476737745025037
22.390035761770147
22.33268668132845
22.290302525686375
22.25712185547666
22.2301807126057
22.207888689158988


In [42]:
loss = test_epoch(dataloader_test, model, nn.MSELoss())
print(loss)

  0%|          | 0/1819 [00:00<?, ?it/s]

22.997070504912575


In [45]:
x_test_sample, y_test_sample = next(iter(dataloader_test))
model(x_test_sample)

tensor([[-1.0255e-02,  2.2910e-01,  3.0603e-02,  ..., -6.2798e-05,
         -1.7010e-03, -5.0732e-03],
        [ 2.9298e-01,  1.0642e+00,  5.6346e-02,  ..., -2.4096e-04,
          2.9729e-03, -8.6043e-02],
        [-1.4080e-02,  2.1203e-01,  4.0611e-02,  ..., -2.5963e-05,
         -1.9898e-03, -3.3154e-03],
        ...,
        [ 4.2368e-02,  7.6389e-02,  3.6934e-02,  ...,  9.6757e-06,
         -4.9688e-04, -2.0190e-03],
        [ 7.7588e-02,  1.4226e-01,  4.5386e-02,  ..., -3.3318e-04,
          6.5945e-04, -8.4444e-04],
        [ 1.5239e-01,  1.6373e+00,  6.6208e-02,  ..., -1.0677e-04,
         -5.8723e-04, -1.0875e-02]], grad_fn=<AddmmBackward0>)